In [33]:
path = 'data/emails.txt'
with open(path, 'rb') as f:
    text = f.read().decode('utf-8', errors='ignore')

In [34]:
import re

def remove_html_tags(text_with_html):
    # Define a regular expression pattern to match HTML tags
    html_tag_pattern = re.compile('<.*?>')

    # Use the `sub` method to replace all matched HTML tags with an empty string
    text_without_html = re.sub(html_tag_pattern, '', text_with_html)

    return text_without_html

text = remove_html_tags(text)

In [35]:
import regex

def cleanup(dirty_text):

    # Define a regular expression pattern to match nested square brackets
    square_brackets_pattern = regex.compile(r'\[(?:[^[\]]*|(?R))*\]')
    dirty_text = regex.sub(square_brackets_pattern, '', dirty_text)

    # Define a regular expression pattern to match text inside square brackets
    square_brackets_pattern = re.compile(r'\[.*?\]')
    dirty_text = re.sub(square_brackets_pattern, '', dirty_text)

    # Weird character
    dirty_text = dirty_text.replace("=E2=80=99", "`")

    # Define a regular expression pattern to match '=E2=80=' followed by a number
    e2_80_sequence_pattern = re.compile(r'=E2=80=\d+')
    dirty_text = re.sub(e2_80_sequence_pattern, '', dirty_text)

    # Define a regular expression pattern to match sequences starting with '=' and ending with a number
    equals_sequence_pattern = re.compile(r'=([A-F0-9]{2}=)+\d+')
    dirty_text = re.sub(equals_sequence_pattern, '', dirty_text)

    # Define a regular expression pattern to match '=' followed by any non-letter characters
    equals_non_letters_pattern = re.compile('=([^a-zA-Z]+)')
    dirty_text = re.sub(equals_non_letters_pattern, '', dirty_text)

    clean_text = dirty_text.replace("=E2", "`")

    return clean_text

def remove_minute_read(text):
    # Define a regular expression pattern to match the sequence
    minute_read_pattern = re.compile(r'\(?\d*\s*MINUTE\s*READ\)?', re.IGNORECASE)

    # Use the `sub` method to replace all matched sequences with an empty string
    clean_text = re.sub(minute_read_pattern, '', text)

    return clean_text

text = cleanup(text)
text = remove_minute_read(text)

In [36]:
arguments = [
    "BIG TECH & STARTUPS",
    "SCIENCE & FUTURISTIC TECHNOLOGY",
    "PROGRAMMING, DESIGN & DATA SCIENCE",
    "MISCELLANEOUS"
]

def extract_sections(text, start_section, end_section):
    # Define the regular expression pattern to match the desired sections
    pattern = re.compile(r'(?i){}(.*?)(?={})'.format(re.escape(start_section), re.escape(end_section)), re.MULTILINE | re.DOTALL)
    sections = re.findall(pattern, text)

    return sections

In [37]:
text_by_argument = {}

# Iterate through the sections_to_extract list using indices
for i in range(len(arguments) - 1):
    argument = arguments[i]
    next_argument = arguments[i + 1]

    text_by_argument[argument] = extract_sections(text, argument, next_argument)

for section, content in text_by_argument.items():
    print(f"{section}: {len(content)} sections extracted")

BIG TECH & STARTUPS: 213 sections extracted
SCIENCE & FUTURISTIC TECHNOLOGY: 212 sections extracted
PROGRAMMING, DESIGN & DATA SCIENCE: 240 sections extracted


In [38]:
def print_first_10_sections(sections):
    for i, section in enumerate(sections[:10], start=1):
        print(section)

#print the first 10 extracted section for the BIG TECH & STARTUPS argument
#print_first_10_sections(text_by_argument[arguments[0]])
print(text_by_argument[arguments[0]][0])



BIG NEWS FROM MICROSOFT CTO ANDREAS BRAUN 



Microsoft CTO Andreas Braun announced that GPT-4 will be released as
early as next week. The new model will be multimodal and will have
video capabilities. The announcement was made during the 'AI in Focus
- Digital Kickoff' hybrid event held on March 9 whereMicrosoft
Germany employees presented LLMs like the GPT series.GOOGLE DUSTS OFF THE FAILED GOOGLE+ PLAYBOOK TO FIGHT CHATGPT 



Google was threatened by Facebook's rise to success in 2011. This
prompted thecompany to issue a decree to its employees to build
social features intoeverything, with their bonuses tied to Google's
social success. The decree resulted in ham-fisted social integrations
across Google that users despised, such as YouTube comments being tied
to Google+ and the real name policy. Just like with the company's
social panic, its current ChatGPT panic may see employees' ratings and
reviews for promotions influenced by their ability to integrate
artificial intelligence 

In [39]:
def download_from_url(url, filepath='temp'):
  import requests
  r = requests.get(url)
  with open(filepath, 'wb') as f:
      f.write(r.content)

In [40]:
def clean_companies(companies):
    # Convert companies in string 
    for i,company in enumerate(companies):
        companies[i] = company.get_text().lower()

    # Append Bigh Techs in parentheses and cleaning
    # Alphabet (Google) --> Alphabet, Google
    for i,company in enumerate(companies):
        if('(' in company):
            companies.append(company[company.find('(')+1:company.find(')')])
            companies[i] = company[0:company.find('(')-1]
    return companies

In [41]:
def get_companies():
    from bs4 import BeautifulSoup

    url = 'https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/'

    download_from_url(url)
    html_text = open('temp', 'r', encoding="utf-8").read()

    soup = BeautifulSoup(html_text, 'html.parser')

    # Get the first 10 companies in tags
    companies = soup.findAll('div', {'class':'company-name'})[0:10]

    return clean_companies(companies)

In [42]:
#Da mappare con arguments (?)
science_kw = [    
    'science', 'mathematics', 'physics', 'biology', 'scientific studies', 
    'astronomy', 'chemistry', 'botany', 'genetics', 'zoology', 
    'physiology', 'earth', 'geology', 'meteo','ecology', 'research']

programming_kw = [
    'programming', 'framework', 'repo', 'repository', 'tool', 'api', 
    'html', 'css', 'js', 'javascript', 'java', 'pyhton', 'c++', 'c#', 
    'design', 'data','data science', 'git', 'sql', 'db', 'database']

In [43]:
import nltk
nltk.download('punkt')

def tokenize(text):
    # Tokenization
    tokens = nltk.word_tokenize(text)
    # print("Numero di tokens:", len(tokens))
    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
def contains_keywords(section, keywords):
    for token in tokenize(section):
        for keyword in keywords:
            if keyword.lower() in token.lower():
                return True
    return False

print("La section è Big Tech?", contains_keywords(text_by_argument[arguments[0]][0], get_companies()))
print("La section è Science?", contains_keywords(text_by_argument[arguments[0]][0], science_kw))
print("La section è Programming?", contains_keywords(text_by_argument[arguments[0]][0], science_kw))

La section è Big Tech? True
La section è Science? False
La section è Programming? False


In [45]:
def get_argument(section):
    if(contains_keywords(section,get_companies())):
        return arguments[0]
    elif(contains_keywords(section,science_kw)):
        return arguments[1]
    elif(contains_keywords(section,programming_kw)):
        return argument[2]
    return arguments[3]

print(get_argument(text_by_argument[arguments[0]][0]))

BIG TECH & STARTUPS


In [46]:
def get_header(section):
    header = ''
    for token in tokenize(section):
        if not token.isupper():
            header = header[:len(header)-1]
            return header
        header += token+' '

print(get_header(text_by_argument[arguments[0]][0]))

BIG NEWS FROM MICROSOFT CTO ANDREAS BRAUN


In [47]:
def show_dataFrame(sections):
    import pandas as pd

    col1 ='section header'
    col2 = 'section argument'

    data_set = { col1 : [], col2 : [] }

    for single_section in sections:
        data_set[col1].append(get_header(single_section))
        data_set[col2].append(get_argument(single_section))
   
    df = pd.DataFrame(data_set)
    return df

In [48]:
a = []
for t in text_by_argument[arguments[0]][:10]:
    a.append(t)

show_dataFrame(a)

,section header,section argument
0,BIG NEWS FROM MICROSOFT CTO ANDREAS BRAUN,BIG TECH & STARTUPS
1,ELON MUSK WASTES NO TIME CHANGING TWITTER,BIG TECH & STARTUPS
2,ELON MUSK PLANNING VINE REBOOT AFTER TWITTER A...,BIG TECH & STARTUPS
3,RIP GOOGLE HANGOUTS,BIG TECH & STARTUPS
4,ELON MUSK BARS TWITTER EMPLOYEES FROM THE COMP...,BIG TECH & STARTUPS
5,FACEBOOK PARENT META IS PREPARING TO NOTIFY EM...,BIG TECH & STARTUPS
6,TWITTER TELLS ADVERTISERSTHAT USER GROWTH IS A...,SCIENCE & FUTURISTIC TECHNOLOGY
7,ELON MUSK HAS DISCUSSED PUTTINGALL OF TWITTER ...,O
8,META LAYING OFF MORE THANTHOUSAND EMPLOYEES,MISCELLANEOUS
9,READ ELON MUSK 'S FIRST EMAIL TOALL TWITTER EM...,BIG TECH & STARTUPS
